In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import lightgbm as lgb
import altair as alt
from altair.expr import datum

In [2]:
train = pd.read_csv(r'D:\HPDemo\Projects\Kaggle_house_price\train.csv')
test = pd.read_csv(r'D:\HPDemo\Projects\Kaggle_house_price\test.csv')

In [3]:
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

The train data size before dropping Id feature is : (1460, 81) 
The test data size before dropping Id feature is : (1459, 80) 

The train data size after dropping Id feature is : (1460, 80) 
The test data size after dropping Id feature is : (1459, 79) 


In [4]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

In [5]:
train["SalePrice"] = np.log1p(train["SalePrice"])

ntrain = train.shape[0]
ntest = test.shape[0]
y_train = train.SalePrice.values
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2917, 79)


D:\Anaconda\envs\classic_jl_lgbm\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [6]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

,Missing Ratio
PoolQC,99.691464
MiscFeature,96.400411
Alley,93.212204
Fence,80.425094
FireplaceQu,48.680151
LotFrontage,16.660953
GarageQual,5.450806
GarageCond,5.450806
GarageFinish,5.450806
GarageYrBlt,5.450806


In [7]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})

In [8]:
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))

all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: x.dropna().skew()).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

Shape all_data: (2917, 78)

Skew in numerical features: 



,Skew
MiscVal,21.950962
PoolArea,17.697766
LotArea,13.116240
LowQualFinSF,12.090757
3SsnPorch,11.377932
LandSlope,4.975813
KitchenAbvGr,4.302763
BsmtFinSF2,4.146636
EnclosedPorch,4.004404
ScreenPorch,3.947131


In [9]:
skewness = skewness[abs(skewness.Skew) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

There are 36 skewed numerical features to Box Cox transform


In [10]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

(2917, 220)


In [11]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [12]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

D:\Anaconda\envs\classic_jl_lgbm\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [14]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.005, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [15]:
%%time
score_1 = rmsle_cv(lasso)
score_2 = rmsle_cv(ENet)
score_3 = rmsle_cv(KRR)
#score_4 = rmsle_cv(GBoost)
score_5 = rmsle_cv(model_lgb)

Wall time: 3.06 s


In [16]:
print(score_1.mean(), score_2.mean(), score_3.mean(), score_5.mean())

0.11027227044577288 0.11024647070496787 0.17803703736736706 0.14265822667278366


In [17]:
model_lgb.fit(train, y_train)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=5, bagging_seed=9,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.2319, feature_fraction_seed=9,
       learning_rate=0.005, max_bin=55, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_data_in_leaf=6, min_split_gain=0.0,
       min_sum_hessian_in_leaf=11, n_estimators=720, n_jobs=-1,
       num_leaves=5, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [18]:
pred = model_lgb.predict(test)
pred = np.exp(pred)
solution = pd.DataFrame(pred, index=test_ID, columns=['SalePrice'])
solution.to_csv('submission_one.csv', index_label='Id', header=True)

In [14]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
    
    def fit(self, X, y):
        self.models_ = [x for x in models]
        
        for model in self.models_:
            model.fit(X, y)
        
        return self
    
    def predict(self, X):
        prediction = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        
        return np.mean(prediction, axis=1)

In [20]:
%%time
models = (ENet, KRR, lasso)
averaged_models = AveragingModels(models=models)

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.1185 (0.0075)

Wall time: 1.17 s


In [15]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        oof = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                oof[holdout_index, i] = y_pred
            
        self.meta_model_.fit(oof, y)
        return self
    
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([
                model.predict(X) for model in base_models]).mean(axis=1)
                for base_models in self.base_models_
        ])
        return self.meta_model_.predict(meta_features)

In [22]:
%%time
stacked_averaged_models = StackingAveragedModels(base_models=(ENet, KRR), meta_model=lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

Stacking Averaged models score: 0.1101 (0.0078)
Wall time: 3.1 s


In [16]:
from sklearn.linear_model import Ridge, Lasso, MultiTaskLasso, ElasticNet, BayesianRidge, LassoLars, SGDRegressor, HuberRegressor
from sklearn.svm import SVR, NuSVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error

In [17]:
clf_ridge = Ridge(random_state=42)
clf_lasso = Lasso(random_state=42)
clf_Enet = ElasticNet(random_state=42)
clf_Bayes = BayesianRidge()
clf_lassolars = LassoLars()
clf_sgd = SGDRegressor()
clf_huber = HuberRegressor()
clf_SVR = SVR()
clf_nuSVR = NuSVR()
clf_knnr = KNeighborsRegressor()
clf_tree = DecisionTreeRegressor()

classifiers = [clf_ridge, clf_lasso, clf_Enet, clf_Bayes, clf_lassolars, clf_sgd, clf_huber, clf_SVR, clf_nuSVR, clf_knnr, clf_tree]

params = {
    0: {'clf__alpha': np.linspace(0.01, 1, 10)},
    1: {'clf__alpha': np.linspace(0.01, 1, 100)},
    2: {'clf__alpha': np.linspace(0.01, 1, 10), 'clf__l1_ratio': np.linspace(0.01, 1, 10)},
    3: {'clf__alpha_1': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4], 'clf__alpha_2': [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]},
    4: {'clf__alpha': np.linspace(0.01, 1, 100)},
    5: {'clf__alpha': np.linspace(0.01, 1, 50), 'clf__l1_ratio': np.linspace(0.01, 1, 50)},
    6: {'clf__alpha': np.linspace(0.01, 1, 50), 'clf__epsilon': np.linspace(1, 2, 10)},
    7: {'clf__kernel' : ['rbf', 'linear', 'poly', 'sigmoid',], 'clf__C': np.linspace(0.1, 1, 6), 'clf__epsilon': np.linspace(0.1, 1, 6)},
    8: {'clf__kernel' : ['rbf', 'linear', 'poly', 'sigmoid'], 'clf__nu': np.linspace(0.1, 1, 6)},
    9: {'clf__n_neighbors': [9], 'clf__weights': ['distance']},
    10: {'clf__max_depth': [5, 6, 8, 10, 12, 20], 'clf__min_samples_split': [2, 3, 5, 10, 20, 30], 'clf__min_samples_leaf': [2, 3, 5, 10, 20, 30]}
}

In [18]:
pca = PCA(n_components=0.9999)
pca.fit(train)
pca_train = pca.transform(train)
pca_train.shape

(1458, 19)

In [155]:
%%time
best_params = {}

for ix, clf in enumerate(classifiers):
    pipeline = Pipeline([
        ('scaler', RobustScaler()),
        ('clf', clf)
    ])
    
    CV = GridSearchCV(pipeline, param_grid=params[ix], scoring='neg_mean_squared_error', verbose=True, n_jobs=-1)
    CV.fit(pca_train, y_train)
    best_params[classifiers[ix]] = CV.best_params_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    3.0s finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    3.4s finished


Fitting 3 folds for each of 100 candidates, totalling 300 fits


...

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.1s finished


Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Done  95 tasks      | elapsed:    3.2s


Wall time: 1min 59s


[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:    7.3s finished


In [157]:
best_params[classifiers[1]]

{'clf__alpha': 0.01}

In [19]:
clf_ridge = Ridge(random_state=42, alpha=1.0)
clf_lasso = Lasso(random_state=42, alpha=0.01)
clf_Enet = ElasticNet(random_state=42, alpha=0.01, l1_ratio=0.23)
clf_Bayes = BayesianRidge(alpha_1=1e-08, alpha_2=0.0001)
clf_lassolars = LassoLars(alpha=0.01)
clf_sgd = SGDRegressor(alpha=0.09081632653061224, l1_ratio=0.5151020408163265)
clf_huber = HuberRegressor(alpha=0.0001, epsilon=4.2)
clf_SVR = SVR(kernel='linear', C=0.82, epsilon=0.1)
clf_nuSVR = NuSVR(kernel='linear', nu=0.45999999999999996)
clf_knnr = KNeighborsRegressor(n_neighbors=9, weights='distance')
clf_tree = DecisionTreeRegressor(max_depth=12, min_samples_leaf=20, min_samples_split=10)
clf_gb = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [25]:
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf, verbose=True, n_jobs=-1))
    return(rmse)

In [ ]:
%%time
score_1 = rmsle_cv(clf_ridge)
score_2 = rmsle_cv(clf_lasso)
score_3 = rmsle_cv(clf_Enet)
score_4 = rmsle_cv(clf_Bayes)
score_5 = rmsle_cv(clf_lassolars)
#score_6 = rmsle_cv(clf_sgd)
score_7 = rmsle_cv(clf_huber)
score_8 = rmsle_cv(clf_SVR)
score_9 = rmsle_cv(clf_nuSVR)
score_10 = rmsle_cv(clf_knnr)
score_11 = rmsle_cv(clf_tree)
score_12 = rmsle_cv(clf_gb)

scores = ['scores_' + x for x in range(13)]

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.6s finished


In [24]:
print(score_1.mean(), score_2.mean(), score_3.mean(), score_4.mean(), score_5.mean(), score_10.mean(), score_11.mean(), score_12.mean())

0.16080112411425276 0.16071743192601817 0.1606740956646162 0.16078689302204777 0.3994682097577765 0.2318819952205305 0.23488593566299315 0.16913801653625954
